# UCF-Crime Anomaly Detection with Multi-Task Learning

This Jupyter Notebook implements a Multi-Task Learning (MTL) pipeline for anomaly detection using an image-based UCF-Crime dataset. The pipeline processes `.png` images, trains a model with four tasks (general anomaly detection, violence detection, property crime detection, anomaly type classification), analyzes task relationships, and conducts an ablation study. The dataset is assumed to be at `/home/user/ucf_crime_dataset` with generated annotation files.

## Setup Instructions
1. Install dependencies: `pip install tensorflow opencv-python numpy pandas scikit-learn scipy`
2. Download the Kaggle dataset: https://www.kaggle.com/datasets/mission-ai/crimeucfdataset
3. Extract `.png` images or use `extract_frames.py` to convert videos to images.
4. Update paths in cells as needed (e.g., `data_dir`).
5. Run cells sequentially, inspecting outputs for debugging.


# Step 1

In [5]:
# Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import numpy as np
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import pearsonr
import csv
import time



In [2]:
# Define constants
DATA_DIR = 'D:/Users/eniang.eniang/Desktop/Multi-task learning/data/'  # Update to your dataset path
TRAIN_ANNOTATION_FILE = 'train_annotations.txt'
TEST_ANNOTATION_FILE = 'test_annotations.txt'
IMAGE_SIZE = (224, 224)  # Image size for ResNet50
BATCH_SIZE = 32
EPOCHS = 10
NUM_ANOMALY_TYPES = 14  # 13 anomaly classes + Normal

# Verify dataset directory
if not os.path.exists(DATA_DIR):
    raise ValueError(f"Dataset directory {DATA_DIR} does not exist. Update DATA_DIR.")

# check train and test directories
train_dir = os.path.join(DATA_DIR, "Train")
test_dir = os.path.join(DATA_DIR, "Test")

print("Train subfolders: ", os.listdir(train_dir) if os.path.exists(train_dir) else "Not found")
print("Test subfolders: ", os.listdir(test_dir) if os.path.exists(test_dir) else "Not found")

print("Setup complete. Dataset directory:", DATA_DIR)



Train subfolders:  ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'NormalVideos', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
Test subfolders:  ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'NormalVideos', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
Setup complete. Dataset directory: D:/Users/eniang.eniang/Desktop/Multi-task learning/data/


# Stage 2: Annotation Generation
Generate annotation files (train_annotations.txt, test_annotations.txt) for the image-based dataset, mapping .png images to class labels based on folder structure.



In [3]:
import glob

def generate_annotation_file(dataset_root, split, output_file):
    """
    Generate an annotation file for a given dataset split (train or test) with .png images.
    """
    split_dir = os.path.join(dataset_root, split)
    if not os.path.exists(split_dir):
        raise ValueError(f"Directory {split_dir} does not exist.")
    
    annotations = []
    class_counts = {}

    # List the class subfolders
    class_names = [name for name in os.listdir(split_dir) if os.path.isdir(os.path.join(split_dir, name))]
    print(f"Found {len(class_names)} class subfolders in {split_dir}: {class_names}")

    for class_name in class_names:
        class_dir = os.path.join(split_dir, class_name)

        # Find the images with multiple extensions
        image_files = []
        for ext in ["*.jpg", "*.PNG", "*.png", "*.JPG"]:
            image_files.extend(glob.glob(os.path.join(class_dir, ext)))

        class_counts[class_name] = len(image_files)

        for image_path in image_files:
            # construct relative path
            relative_path = os.path.join(split, class_name, os.path.basename(image_path))
            # Quote path to handle spaces
            annotation = f'"{relative_path}" {class_name}'
            annotations.append(annotation)

    # Write the annotations
    with open(output_file, "w") as f:
        for annotation in annotations:
            f.write(annotation + '\n')
    
    # debugging output
    print(f"Generated {output_file} with {len(annotations)} entries")
    print(f"Images per class in {split}:", class_counts)
    if annotations:
        print("Sample annotations (first 5):", annotations[:5])
    if not annotations:
        print("WARNING!!! no images found")
    
    return class_counts


# Generate Annotations
print("Generating training annotations..........")
train_class_counts = generate_annotation_file(DATA_DIR, "Train", TRAIN_ANNOTATION_FILE)
print("Generating test annotations..............")
test_class_counts = generate_annotation_file(DATA_DIR, "Test", TEST_ANNOTATION_FILE)

# verify annotation files
if os.path.exists(TRAIN_ANNOTATION_FILE) and os.path.exists(TEST_ANNOTATION_FILE):
    print("\nAnnotation files created successfully.")
    print("First 5 lines of the train_annotation.txt:")
    with open(TRAIN_ANNOTATION_FILE, 'r') as f:
        print(f.readlines()[:5])

else:
    raise FileNotFoundError("Annotation file not found. Check generation process")

Generating training annotations..........
Found 14 class subfolders in D:/Users/eniang.eniang/Desktop/Multi-task learning/data/Train: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'NormalVideos', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
Generated train_annotations.txt with 2532690 entries
Images per class in Train: {'Abuse': 38152, 'Arrest': 52794, 'Arson': 48842, 'Assault': 20720, 'Burglary': 79008, 'Explosion': 37506, 'Fighting': 49368, 'NormalVideos': 1895536, 'RoadAccidents': 46972, 'Robbery': 82986, 'Shooting': 14280, 'Shoplifting': 49670, 'Stealing': 89604, 'Vandalism': 27252}
Sample annotations (first 5): ['"Train\\Abuse\\Abuse001_x264_0.png" Abuse', '"Train\\Abuse\\Abuse001_x264_10.png" Abuse', '"Train\\Abuse\\Abuse001_x264_100.png" Abuse', '"Train\\Abuse\\Abuse001_x264_1000.png" Abuse', '"Train\\Abuse\\Abuse001_x264_1010.png" Abuse']
Generating test annotations..............
Found 14 class subfolders in D:/U

# Stage 3: Data Loading
Load .png images and labels from the annotation files using load_ucf_crime_data. Outputs the number of loaded images and label shapes for debugging.



In [12]:
# Stage 3: Data Loading
def load_ucf_crime_data(data_dir, annotation_file, image_size=(224, 224), batch_size=32):
    start_time = time.time()

    # verify the annotation file exists
    if not os.path.exists(annotation_file):
        raise FileNotFoundError(f"Annotation file {annotation_file}  not found")
    
    # Read annotation file
    try:
        annotations = pd.read_csv(annotation_file, sep=' ', header=None,
        names=["image", "label"], quotechar='"')
        print(f"Loaded annotation file: {annotation_file}")
        print(f"First 5 entries: {annotations.head()}")
        print(f"The total annotations: {len(annotations)}")
    
    except pd.errors.ParserError as e:
        print(f"Error parsing {annotation_file}: {e}")
        print("Check if file has spaces in filenames or incorrect delimiters.")
        raise

    # Define anomaly classes
    anomaly_classes = tf.constant(['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting','NormalVideos', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism'])
    violent_classes = tf.constant(["Assault", "Fighting", "Shooting"])
    property_classes = tf.constant(["Burglary", "Stealing", "Shoplifting", "Vandalism"])

    @tf.function
    def process_path(image_path, label):
        # Load and preprocess image
        img = tf.io.read_file(image_path)
        img = tf.image.decode_png(img,channels=3)
        img = tf.image.resize(img, image_size)
        img = img / 255.0
        img = tf.cast(img, tf.float32)

        # Assign labels
        is_anomaly = tf.reduce_any(tf.equal(anomaly_classes, label))
        is_anomaly = tf.cast(is_anomaly, tf.int32)

        is_violent = tf.reduce_any(tf.equal(violent_classes, label))
        is_violent = tf.cast(is_violent, tf.int32)

        is_property = tf.reduce_any(tf.equal(property_classes, label))
        is_property = tf.cast(is_property, tf.int32)

        # Find index of label in anomaly_classes, or len(anomaly_classes) if not found
        matches = tf.equal(anomaly_classes, label)
        anomaly_type = tf.reduce_max(tf.where(matches))
        anomaly_type = tf.where(tf.reduce_any(matches), anomaly_type, tf.cast(len(anomaly_classes),tf.int32))
        anomaly_type = tf.cast(anomaly_type, tf.int32)

        return img, {
            "general_anomaly": is_anomaly,
            "violence": is_violent,
            "property_crime":is_property,
            "anomaly_type": anomaly_type
        }

    # Create tf.data.Dataset
    image_path = [os.path.normpath(os.path.join(data_dir, row['image'])) for _, row in annotations.iterrows()]
    labels = annotations['label'].tolist()

    # Filter valid path
    valid_indices = [ i for i, path in enumerate(image_path) if os.path.exists(path)]
    if len(valid_indices) < len(image_path):
        print(f"WARNING!!: {len(image_path) - len(valid_indices)} invalid image paths found")
        print("Sample invalid paths:", [image_path[i] for i in range(len(image_path)) if i not in valid_indices][:5])

    image_path = [image_path[i] for i in valid_indices]
    labels = [labels[i] for i in valid_indices]
    print(f"Valid images: {len(image_path)}")

    if not image_path:
        print("ERROR: No valid images found.")

    dataset = tf.data.Dataset.from_tensor_slices((image_path, labels))
    dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    # Collect images and labels
    images = []
    label_dict = {
        'general_anomaly':[],
        "violence": [],
        "property_crime":[],
        "anomaly_type": []
    }


    loaded_counts = 0
    for batch_images, batch_labels in dataset:
        images.append(batch_image.numpy())
        for task in label_dict:
            label_dict[task].append(batch_labels[task].numpy())
        loaded_counts += len(batch_images)
        print(f"Processed {loaded_counts} images....", end='\r')

    images = np.concatenate(images, axis=0)
    for task in label_dict:
        label_dict[task] = np.concatenate(label_dict[task], axis=0)

    # Debugging output
    print(f"\nLoaded {loaded_counts} images from {annotation_file}")
    print("Image array shape:", images.shape)
    print("Label shapes: {task: label_dict[task].shape for task in label_dict}")
    print(f"Time taken: {time.time() - start_time:.2f} seconds")


# Load data
print("Loading training data .....")
X_train, y_train = load_ucf_crime_data(DATA_DIR, TRAIN_ANNOTATION_FILE, IMAGE_SIZE, BATCH_SIZE)
print("\nLoading validation data............")
X_val, y_val = load_ucf_crime_data(DATA_DIR, TEST_ANNOTATION_FILE, IMAGE_SIZE, BATCH_SIZE)

# verify data shapes
print("\nTraining images shape:", X_train.shape)
print("Training labels:", {task: y_train[task].shape for task in y_train})
print("Validation images shape:", X_val.shape)
print("Validation labels:", {task: y_val[task].shape for task in y_val})





Loading training data .....
Loaded annotation file: train_annotations.txt
First 5 entries:                                 image  label
0     Train\Abuse\Abuse001_x264_0.png  Abuse
1    Train\Abuse\Abuse001_x264_10.png  Abuse
2   Train\Abuse\Abuse001_x264_100.png  Abuse
3  Train\Abuse\Abuse001_x264_1000.png  Abuse
4  Train\Abuse\Abuse001_x264_1010.png  Abuse
The total annotations: 2532690
Valid images: 2532690


TypeError: in user code:

    File "D:\Users\eniang.eniang\AppData\Local\Temp\2\ipykernel_5628\2460950749.py", line 49, in process_path  *
        anomaly_type = tf.where(tf.reduce_any(matches), anomaly_type, tf.cast(len(anomaly_classes),tf.int32))

    TypeError: Input 'e' of 'SelectV2' Op has type int32 that does not match type int64 of argument 't'.


# Stage 4: Model Definition
Define the MTL model with a ResNet50 backbone and four task-specific heads using build_mtl_model. Display the model summary for inspection.



In [ ]:
def build_mtl_model(input_shape=(224, 224, 3), num_anomaly_types=14):
    """
    Build MTL model with shared ResNet50 backbone and task-specific heads for images.
    """
    inputs = tf.keras.Input(shape=input_shape)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    
    x = base_model(inputs)
    pooled = layers.GlobalAveragePooling2D()(x)
    shared_dense = layers.Dense(512, activation='relu')(pooled)
    
    anomaly_output = layers.Dense(1, activation='sigmoid', name='general_anomaly')(shared_dense)
    violence_output = layers.Dense(1, activation='sigmoid', name='violence')(shared_dense)
    property_output = layers.Dense(1, activation='sigmoid', name='property_crime')(shared_dense)
    type_output = layers.Dense(num_anomaly_types, activation='softmax', name='anomaly_type')(shared_dense)
    
    model = models.Model(inputs, [
        anomaly_output, violence_output, property_output, type_output
    ])
    
    return model

# Build model and display summary
model = build_mtl_model(input_shape=(224, 224, 3), num_anomaly_types=NUM_ANOMALY_TYPES)
model.summary()

# Stage 5: Task Relationship Functions
Define functions to analyze task relationships: compute_gradient_alignment (cosine similarity of gradients) and compute_loss_correlation (Pearson correlation of losses).



In [ ]:
def compute_gradient_alignment(model, data, labels, task_names):
    """
    Compute cosine similarity between gradients of tasks.
    """
    gradients = {}
    for task in task_names:
        with tf.GradientTape() as tape:
            predictions = model(data)
            task_idx = task_names.index(task)
            loss = tf.keras.losses.binary_crossentropy(labels[task], predictions[task_idx])
            if task == 'anomaly_type':
                loss = tf.keras.losses.sparse_categorical_crossentropy(labels[task], predictions[task_idx])
        grads = tape.gradient(loss, model.trainable_variables)
        gradients[task] = grads
    
    similarities = {}
    for task1 in task_names:
        for task2 in task_names:
            if task1 >= task2:
                continue
            g1 = tf.concat([tf.reshape(g, [-1]) for g in gradients[task1] if g is not None], axis=0)
            g2 = tf.concat([tf.reshape(g, [-1]) for g in gradients[task2] if g is not None], axis=0)
            cos_sim = tf.reduce_sum(g1 * g2) / (tf.norm(g1) * tf.norm(g2))
            similarities[f'{task1}_vs_{task2}'] = cos_sim.numpy()
    
    print("Computed gradient similarities:", similarities)
    return similarities

def compute_loss_correlation(losses_dict, task_names):
    """
    Compute Pearson correlation between task losses.
    """
    correlations = {}
    for task1 in task_names:
        for task2 in task_names:
            if task1 >= task2:
                continue
            corr, _ = pearsonr(losses_dict[task1], losses_dict[task2])
            correlations[f'{task1}_vs_{task2}'] = corr
    print("Computed loss correlations:", correlations)
    return correlations

# Stage 6: Training Function
Define train_mtl_model to train the MTL model, compute task relationships, and return training history. Monitor training progress via printed outputs.



In [ ]:
def train_mtl_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32):
    """
    Train MTL model and collect losses for correlation analysis.
    """
    task_names = ['general_anomaly', 'violence', 'property_crime', 'anomaly_type']
    losses_dict = {task: [] for task in task_names}
    
    model.compile(
        optimizer='adam',
        loss={
            'general_anomaly': 'binary_crossentropy',
            'violence': 'binary_crossentropy',
            'property_crime': 'binary_crossentropy',
            'anomaly_type': 'sparse_categorical_crossentropy'
        },
        loss_weights={
            'general_anomaly': 1.0,
            'violence': 0.5,
            'property_crime': 0.5,
            'anomaly_type': 1.0
        },
        metrics={
            'general_anomaly': ['accuracy', tf.keras.metrics.AUC(name='auc')],
            'violence': ['accuracy'],
            'property_crime': ['accuracy'],
            'anomaly_type': ['accuracy']
        }
    )
    
    history = model.fit(
        X_train,
        [y_train[task] for task in task_names],
        validation_data=(X_val, [y_val[task] for task in task_names]),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    
    for task in task_names:
        losses_dict[task] = history.history[f'{task}_loss']
    
    grad_similarities = compute_gradient_alignment(model, X_val[:batch_size], y_val, task_names)
    loss_correlations = compute_loss_correlation(losses_dict, task_names)
    
    return history, grad_similarities, loss_correlations

# Train model (uncomment to run after verifying previous stages)
# history, grad_similarities, loss_correlations = train_mtl_model(model, X_train, y_train, X_val, y_val, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Stage 7: Ablation Study
Define ablation_study to evaluate the model with subsets of tasks, reporting AUC for general anomaly detection.



In [ ]:
def ablation_study(X_train, y_train, X_val, y_val, tasks_to_include, input_shape=(224, 224, 3), num_anomaly_types=14):
    """
    Train model with a subset of tasks for ablation study.
    """
    inputs = tf.keras.Input(shape=input_shape)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    
    x = base_model(inputs)
    pooled = layers.GlobalAveragePooling2D()(x)
    shared_dense = layers.Dense(512, activation='relu')(pooled)
    
    outputs = []
    loss_dict = {}
    metrics_dict = {}
    
    for task in tasks_to_include:
        if task in ['general_anomaly', 'violence', 'property_crime']:
            output = layers.Dense(1, activation='sigmoid', name=task)(shared_dense)
            loss_dict[task] = 'binary_crossentropy'
            metrics_dict[task] = ['accuracy']
        elif task == 'anomaly_type':
            output = layers.Dense(num_anomaly_types, activation='softmax', name=task)(shared_dense)
            loss_dict[task] = 'sparse_categorical_crossentropy'
            metrics_dict[task] = ['accuracy']
        outputs.append(output)
    
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer='adam',
        loss=loss_dict,
        metrics=metrics_dict
    )
    
    history = model.fit(
        X_train,
        [y_train[task] for task in tasks_to_include],
        validation_data=(X_val, [y_val[task] for task in tasks_to_include]),
        epochs=5,
        batch_size=32,
        verbose=1
    )
    
    val_preds = model.predict(X_val)
    general_idx = tasks_to_include.index('general_anomaly') if 'general_anomaly' in tasks_to_include else None
    if general_idx is not None:
        auc = roc_auc_score(y_val['general_anomaly'], val_preds[general_idx])
        print(f"AUC for tasks {tasks_to_include}: {auc:.4f}")
        return auc
    return None

# Stage 8: Execution and Results
Run the pipeline, train the model, compute task relationships, and perform the ablation study. Display results for gradient alignment, loss correlation, and AUC scores.



In [ ]:
# Train the model
history, grad_similarities, loss_correlations = train_mtl_model(
    model, X_train, y_train, X_val, y_val, epochs=EPOCHS, batch_size=BATCH_SIZE
)

# Print task relationship results
print("\nGradient Alignment (Cosine Similarity):")
for pair, sim in grad_similarities.items():
    print(f"{pair}: {sim:.4f}")

print("\nLoss Correlation (Pearson):")
for pair, corr in loss_correlations.items():
    print(f"{pair}: {corr:.4f}")

# Perform ablation study
task_combinations = [
    ['general_anomaly'],
    ['general_anomaly', 'violence', 'property_crime'],
    ['general_anomaly', 'anomaly_type'],
    ['general_anomaly', 'violence', 'property_crime', 'anomaly_type']
]

print("\nAblation Study Results (AUC for General Anomaly Detection):")
for tasks in task_combinations:
    auc = ablation_study(X_train, y_train, X_val, y_val, tasks)
    if auc is not None:
        print(f"Tasks: {tasks}, AUC: {auc:.4f}")